# Answer Key to the Data Wrangling with DataFrames Coding Quiz

Helpful resources:
https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html

In [40]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, avg
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType
import pandas as pd

In [19]:
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) write code to answer the quiz questions 

spark = SparkSession.builder.getOrCreate()
df = spark.read.json("music_log_small.json")

# Question 1

Which page did user id "" (empty string) NOT visit?

In [8]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [49]:
df.select(["page"]).where(df.userId == "").dropDuplicates().sort("page").show()


+-----+
| page|
+-----+
|About|
| Help|
| Home|
|Login|
+-----+



In [50]:
page_all = df.select(["page"]).dropDuplicates().sort("page").collect() 
page_id = df.select(["page"]).where(df.userId == "").dropDuplicates().sort("page").collect()

In [51]:
q1 = [ x[0] for x in page_all if x not in page_id] 
pd.DataFrame({"page":q1})

,page
0,Downgrade
1,Error
2,Logout
3,NextSong
4,Save Settings
5,Settings
6,Submit Downgrade
7,Submit Upgrade
8,Upgrade


# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?

In [ ]:
df.select(["userId", "firstname","gender", "page", "song","itemInSession","userAgent"]).where(df.userId == "").show()

+------+---------+------+-----+----+-------------+---------+
|userId|firstname|gender| page|song|itemInSession|userAgent|
+------+---------+------+-----+----+-------------+---------+
|      |     null|  null|Login|null|            0|     null|
|      |     null|  null| Home|null|           26|     null|
|      |     null|  null| Home|null|            5|     null|
|      |     null|  null| Home|null|            5|     null|
|      |     null|  null|Login|null|            6|     null|
|      |     null|  null|Login|null|            0|     null|
|      |     null|  null| Home|null|           14|     null|
|      |     null|  null| Home|null|            7|     null|
|      |     null|  null|Login|null|            0|     null|
|      |     null|  null| Home|null|            0|     null|
|      |     null|  null|Login|null|            1|     null|
|      |     null|  null| Home|null|            0|     null|
|      |     null|  null| Help|null|            1|     null|
|      |     null|  null

It could be the Admin 

# Question 3

How many female users do we have in the data set?

In [58]:
df.select(["userID"]).where(df.gender == "F").dropDuplicates().count()

462

# Question 4

How many songs were played from the most played artist?

In [67]:
df.select(["artist"]).groupBy("artist").count().sort(desc("count")).show(2)

+--------+-----+
|  artist|count|
+--------+-----+
|    null| 1653|
|Coldplay|   83|
+--------+-----+
only showing top 2 rows



# Question 5 

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [84]:
erg=df.select(["userId", "page", "song"]).where(df.page == "NextSong").groupBy("userId").count().agg({"count":"avg"}).withColumnRenamed("avg(count)","avg").collect()[0][0]
print("AVG: "+ str(round(erg))+" Songs pro User")

AVG: 10 Songs pro User
